**The report of assignment03**

**Introduction (!! important !!)**

Before running the script, we need to prepare the dataset of the "top-1m-websites" and the "curl" tool.
To download the dataset, we prepared a script -- "download_bash.sh".
It will download the zip-file and extract the compressed csv-file to the path "./toplist_1m/top-1m.csv".

For compiling the curl with http/3 and http/2, there are two scripts in the folders "./build_curl/h2" and "./build_curl/h3".
Please enter the folders and run these scripts to download and compile the "curl".
The executive bin-files should have the paths:
```
curl_program_h3 = "./build_curl/h3/curl/src/curl"
curl_program_h2 = "./build_curl/h2/curl/src/curl"
```
you can also change the program paths in the "utils.py".


a) Read up on transport layer ossification
to briefly answer the following questions: Why is it difficult to develop
and deploy/adopt new transport protocols? (2 points)

Solution a)

The development and support of a new transport protocol highly depend on
the behaviors of hosts and middleboxes. A new transport protocol should accommodate the middleboxes and hosts deployed today
on the Internet. To develop a new transport protocol with the support of middleboxes,
the design should base on UDP or TCP. The UDP solution needs to implement many features, which are already implemented
in TCP. It is very hard to design those mechanisms.
The extension of TCP causes ossification.
The middleboxes are primarily designed for the current TCP and have limited supports to TCP Options.
We only discuss the solution of TCP extension here.

The challenges of the middlebox accommodation are mainly from the following aspects:
> Modification of Sequence Number: currently, the middleboxes may modify the TCP Sequence Number.
The new transport protocol needs to provide a safe usage of the Sequence Number.

> Sequence Space Holes: some middleboxes do not allow TCP sequence holes, an end-to-end communication
may be only successful with only one direction, not both.

> Proxy: the new transport protocol should consider the proxy transport.

> Inconsistent Retransmission: currently, the middleboxes perform differently with the retransmission of variation packets on different ports.
The new transport protocol should design the inconsistent retransmission with only a tiny variation of the packet size.

> Re-segmentation: the middleboxes may re-segment the packets, which may cause fail validation of MAC.
The new transport protocol should provide a mechanism of the intermediate Re-segmentation.

The challenges of the host side are mainly from the following aspects:

> OS and NIC: OS may be not compatible with the new transport protocol, and the OS may use NIC-offload to improve efficiency,
so the compatibility of NIC is also a challenge because it is a hardware problem.

> Performance bottleneck: the new transport protocol may have some encryption features. They should not cause a performance
problem on both server and client sides.

Thus, it is difficult to develop and deploy/adopt new transport protocols.


b) Read up on QUIC
to briefly answer the following questions: How does QUIC overcome the ossification
issue? Why does it use UDP? What features and benefits does it provide over TCP? (2 points)



Solution b)

QUIC uses UDP to traverse middleboxes, authenticated and encrypted transport to prevent modification and limiting ossification
of the protocol by middleboxes.

In the question a, we discuss the TCP ossification from the middleboxes. Directly using UDP
can avoid the problems of TCP extension, therefore QUIC packets can transfer over current middleboxes.

There are the following features and benefits of using QUIC:

> Low Handshake Delay: it minimizes the setup RTTs, achieves 1-RTT initial handshake, 0-RTT subsequent successful handshake, and
0-RTT subsequent failed handshake.

> Stream Multiplexing: it uses multiple streams within a connection to avoid head-of-line blocking of TCP’s sequential delivery.

> Authentication and Encryption: the packets are fully authenticated and mostly encrypted (unencrypted public header and encrypted body).

> Simpler and more accurate Loss Recovery: new packet numbers for all QUIC packets including retransmission packets to
avoid "retransmission ambiguity" problem in TCP

> Flow Control and Congestion Control: connection-level flow control and stream-level flow control;
a variant of "mulTCP" for Cubic during the congestion avoidance phase of Congestion Control.

> NAT Rebinding and Connection Migration: it uses Connection ID to identify connections

> Do not affect current HTTPS: it uses "Alt-Svc" header in their HTTP responses to infer the QUIC support




c) Download the latest standard Tranco toplist (Top 1M). For this assignment, we will use the Top 10k to
determine the adoption of HTTP/3 (and therefore QUIC) on the Web.

We recommend to not use Wi-Fi for measurements, as results will exhibit much more variance. For
this assignment, we will only look at measurements over IPv4.

Prepend www. to the domains and try establishing HTTP/3 connections to the Top 10k Web domains via curl.
Note that HTTP/3 support is currently experimental in curl, meaning you will have to explicitly enable it while
building curl.

When running curl with the --http3 flag, websites that do not support HTTP/3 will fail instead of falling back
to HTTP/2 as of the current version (7.73.0). Further, use the -w flag of curl in order to record following
metrics for each HTTP/3-enabled website:

• time_appconnect

• time_connect

• time_namelookup

• time_pretransfer

• time_redirect

• time_starttransfer

• time_total

• remote_ip

• remote_port

Make sure to convert the timings to reasonable units (e.g., ms or s), if necessary. Briefly explain what each of
the above metrics represents. Save your measurement results in a CSV file (with domain name + above metrics
as columns). Submit your description, CSV file, and measurement script(s). (2 points)

Solution c)

The measurement was finished by "compute engines of google cloud (europe-west3-c)" to avoid variance of WIFI.

To speed up the measurement process, we divide the whole task into fragments.
Every chunk can be processed in a dedicated machine. The thread number of each machine is limited to 10 to avoid the
interaction of multiple 'curl' processes (bandwidth limit).

We used the dataset of the averaged rankings list over the past 30 days. To start the curl process locally, we used
the python script "question_c.py". The task was finished by loop this time. The result was "./qustion_c_output/output.csv".

```
# download the dataset
sh download_bash.sh
# run python, the output in "./qustion_c_output/"
python ./question_c.py
```

The csv-file also contained a column "status".
Only the measurement with "ok" status was successful, because it is meaningless to keep the measurement time of
the websites with the wrong status, such as "timeout" (HTTP/3 unsupported) and "connection refused" (DNS error). Another
status was "error", which means the website transferred something back but it caused an error of the 'curl' program.

The metrics of HTTP/3 and HTTP/2 are different.

The whole sequence of HTTP/2 will be:
```
DNS: name lookup phase
TCP: TCP handshake/establish phase
TLS: TLS negotiation phase
C: the server is calculating the response (machine response time)
Transfer: transfer the data (content)
|-- 1 --|                                       time_namelookup
|---     2    ---|                              time_connect
|--DNS--||--TCP--||--TLS--||-C-|--Transfer--|   whole process
|---         3         ---|                     time_appconnect
|---         4          ---|                    time_pretransfer
|---            5           ---|                time_starttransfer
|---                  6                  ---|   time_total
```
The metrics of HTTP/2:

> time_namelookup: the time used for DNS lookup, see 1

> time_connect: the time to finish establishing the TCP connection, see 2

> time_appconnect: the time TLS finished, see 3

> time_pretransfer: the time about to begin transferring data, see 4

> time_starttransfer: the time the first byte about to be transferred, see 5

> time_total: the total time, see 6

> time_redirect: the time for all redirection steps



The whole sequence of HTTP/3 will be:
```
DNS: name lookup phase
QUIC: QUIC connect to the remote host (the QUIC initial 1-RTT handshake phase)
C: the server is calculating the response (machine response time)
Transfer: transfer the data (content)
|--- 1 ---|                                     time_namelookup
|---       2      ---|                          time_connect
|---       3       ---|                         time_pretransfer
|---DNS---||---QUIC--||--C--|----Transfer---|   whole process
|---           4         ---|                   time_starttransfer
|---                  5                  ---|   time_total
```
The metrics of HTTP/3:
> time_namelookup: the time used for DNS lookup, see 1

> time_connect: the completion time of the QUIC connection, see 2

> time_appconnect: meaningless, because no TLS handshake phase

> time_pretransfer: the time about to begin transferring data, see 3

> time_starttransfer: the time the first byte about to be transferred, see 4

> time_total: the total time, see 5

> time_redirect: the time for all redirection steps

d) How many of the initial ≈10k websites support HTTP/3? Determine the AS numbers (cf. Assignment 01) for
the IP addresses that you collected. Count the number of websites for each ASN, sort them in descending order,
and export it to a CSV file (columns: asn,num_http3_websites). Which ASNs have the highest number of
HTTP/3-enabled websites, i.e., who is responsible for pushing HTTP/3 adoption? Submit your answers, scripts,
and CSV file. (2 points)


Solution d)
There are only 527 websites that support HTTP/3.

The python script is "question_d.py".
```
# after question c (depends on the question c's result file)
python ./question_d.py
```

The result was "./qustion_d_output/top-1k-h3-asn.csv".
The print result listed here:
```
527 websites support HTTP/3!
      asn  num_http3_websites    provider
0   13335                 319  Cloudflare
1   15169                 186      Google
2   32934                  11    Facebook
...

```
Cloudflare has the highest number of HTTP/3-enabled websites.
Google and Facebook also push HTTP/3 adoption.

e) Order the collected timing metrics by the sequential order of their corresponding events (i.e., DNS lookup
happens before y, etc.). Calculate the starting/end times and duration of the events; note that some metrics
include other metrics, so you will need to calculate deltas in some cases. Draw boxplots over all measurements
for each of the events/metrics (cf. Assignment 02); you can use the description of the events/metrics as labels
for your x-axis (e.g., DNS lookup, ...). Submit your answers/descriptions, analysis and plotting scripts, and plot.
(3 points)


Solution e)

The python script is "question_e.py".
```
# after question c (depends on the question c's result file)
python ./question_e.py
```
The plots were "./qustion_e_output/lim1.png" and "./qustion_e_output/lim8.png"

The whole time sequence can be found in the solution of question c (omit here, see solution c).

According to the plots, there are many outliers. It infers that the internet environment is complex and diverse.
The first phase "DNS lookup" has an average time of about 0.25s, most DNS queries can be finished in 0.25s.
The second phase "QUIC" has an average time of about 0s. The reason may be: we run the task on Google Cloud and
google also has many websites with HTTP/3 on the ranking list. Although the true average time may be larger,
the QUIC connection handshake is still very fast because the third quartile is about 0.25s.
The computing time of the server has a narrow interquartile range. It means the standard deviation may be very large.
Servers and websites are various, some are only simple pages whereas others provide complex apps.
The transferring time is also about 0s. One reason may be "Google" (the same as "QUIC"), another may be:
nowadays, websites are mostly dynamic websites and depend on Javascript, so websites themselves only provide tiny static pages.

f) In order to compare the performance of QUIC+HTTP/3, repeat your curl measurements for the Top 1k
websites which you have found to support HTTP/3 with TCP+TLS+HTTP/2. If you found less than 1k
websites with HTTP/3 support, use this number instead. Extend your scripts from e) so that you show the
distributions for HTTP/2 and HTTP/3 next to each other. Which boxes are similar (i.e., unaffected) and which
boxes are different (i.e., faster/slower) when comparing HTTP/2 and HTTP/3? Submit your extended script(s),
plot, and answers. (2 points)

Solution f)

The python script is "question_f.py".
```
# after question c (depends on the question c's result file)
python ./question_f.py
```

The plots were "./qustion_f_output/lim1.png" and "./qustion_f_output/lim8.png"

Comparing the metrics with HTTP/3 and HTTP/2 in the plot, the "DNS lookup", "QUIC" (TCP/TLS), and "Transfer" phases with HTTP/2 have narrower interquartile ranges, but
the "Compute" phase has a wider interquartile range. The average DNS lookup time is lower. It may be caused by the DNS cache.
As we know, the establishment of a QUIC connection only needs 1-RTT time, so the stability of connection with HTTP/3 time is better.
The average computing time of HTTP/3 is slower than HTTP/2. It means the back-end technology of HTTP/3 is currently still not as mature as HTTP/2.
HTTP/2 has a better average total time now.

g) Calculate the differences in raw timing metrics from curl between TCP+TLS+HTTP/2 and QUIC+HTTP/3
for each of the (max.) 1k websites, i.e., ∆value = valuehttp/2 − valuehttp/3. For each metric, draw a boxplot over
the deltas. Store the 25th, 50th, and 75th percentiles of each box in a CSV file. What are the interquartile range
of the deltas, and what does that mean w.r.t. HTTP/2 and HTTP/3? Submit your analysis/plotting script(s) and
discussion. (2 points)

Solution g)

The python script is "question_g.py".
```
# after question c (depends on the question c's result file)
python ./question_g.py
```

The csv-file was "./qustion_g_output/percentiles.csv"

The plots were "./qustion_g_output/lim1.png" and "./qustion_g_output/lim8.png"

In the plot, all average metrics except "time_app_connect" are less than 0. It means HTTP/2 performs
better than HTTP/3 in general.

h) For the 1k websites with HTTP/3 support, use Selenium with a headless Chrome browser to automate
webpage navigation. Make sure that QUIC is enabled and that the used protocol is HTTP/3 for your measurements.
With your setup, extract webpage timing statistics about responseStart, domInteractive,
and domComplete (see screenshot from Chrome developer tools below; note the nextHopProtocol being
h3-Q050, i.e., HTTP/3 with QUIC Q050) through the aforementioned interfaces for all HTTP/3-enabled websites.
Repeat the measurements for the websites, but disable QUIC this time to use TCP+TLS+HTTP/2 instead. Save
your results in two CSV files (one for HTTP/2, one for HTTP/3), with the domain name + metrics as columns.
Submit your script(s) and CSV files. (3 points)

Solution h)

To run Selenium, we need to download the driver of chrome firstly.

The python script is "question_h.py".
```
# after question f (depends on the question f's result file)
python ./question_h.py
```

The results were "./qustion_h_output/summary_chrome_h2.csv" and "./qustion_h_output/summary_chrome_h3.csv".

i) Draw a CDF figure for both HTTP/2 (dashed lines) and HTTP/3 (solid lines) measurements, with separate
curves for responseStart, domInteractive, and domComplete. What do the different metrics represent?
For some percentiles of your choice, discuss the difference between HTTP/2 and HTTP/3, esp. regarding the
impact of HTTP/3 and QUIC for the user. Submit your plotting script, plot, and discussion. (2 points)

Solution i)

The python script is "question_h.py".
```
# after question h (depends on the question h's result file)
python ./question_i.py
```

The results were "./qustion_i_output/output_no_limit.png" and "./qustion_i_output/output_limit_10000.png".

The plot shows that at the low time domain, HTTP/3 has more sample number with respect to all three metrics.
However at the large time domain, HTTP/2 sample number increases quickly.
The intersection of two curves "domComplete" is at about (3000ms, 0.55), "domInteractive" at about (2000ms, 0.75), and "responseStart" at about (1000ms, 0.95).
It means the response time with HTTP/3 is shorter and the small website can benefit from HTTP/3 because QUIC can establish the connection quickly